# Texter with vanilla cnn

In [1]:
%matplotlib inline
%xmode plain

import os
import numpy as np

from texter.utils import eval_utils as evu
from texter.utils import config_utils as cu
from texter.utils import text_utils as tu

np.random.seed(15101993)

Exception reporting mode: Plain


/home/xenomorph/applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/xenomorph/applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
text_path = "../../../citi/citi_rest/data/raw_data/"
text_labels = os.listdir(text_path)
pretrained_embeddings_path = "../../data/others/glove.6B.50d.txt"
pre_path = "../../data/others/GoogleNews-vectors-negative300-SLIM.bin"

X_train, X_val, Y_train, Y_val, tokenizer, word_index = cu.keras_data_config(text_path, text_labels, 
                                                                    3, NUM_WORDS=1000)

Found 57 unique tokens.
Shape of X train and X validation tensor: (8, 2000) (3, 2000)
Shape of label train and validation tensor: (8, 3) (3, 3)


In [3]:
model = cu.keras_model_config(pretrained_embeddings_path=pretrained_embeddings_path, 
                        word_index=word_index, 
                        num_class=3,
                        MAX_SEQUENCE_LENGTH=2000,
                        EMBEDDING_DIM=50, 
                        embedding_type="glove", 
                        model_type="vanilla")

Found 400000 word embeddings


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 2000, 50)          2900      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1996, 256)         64256     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 399, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 395, 64)           81984     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 79, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 75, 256)           82176     
__________

In [5]:
model.fit(X_train, Y_train, batch_size=256,
          epochs=5, validation_data=(X_val, Y_val), callbacks=evu.add_callbacks())

Train on 8 samples, validate on 3 samples
Epoch 1/5
8/8 [==============================] - 4s 507ms/step - loss: 1.0999 - acc: 0.1250 - val_loss: 1.0989 - val_acc: 0.3333
Epoch 2/5
8/8 [==============================] - 0s 11ms/step - loss: 1.0960 - acc: 0.3750 - val_loss: 1.1089 - val_acc: 0.3333
Epoch 3/5
8/8 [==============================] - 0s 11ms/step - loss: 1.0913 - acc: 0.3750 - val_loss: 1.1220 - val_acc: 0.3333
Epoch 4/5
8/8 [==============================] - 0s 12ms/step - loss: 1.0854 - acc: 0.3750 - val_loss: 1.1373 - val_acc: 0.3333
Epoch 5/5
8/8 [==============================] - 0s 12ms/step - loss: 1.0782 - acc: 0.3750 - val_loss: 1.1570 - val_acc: 0.3333


In [6]:
#sample predictions

s = """Ayahuasca emerged again in the early 1960s with the counterculture movement.\n 
Beat writers like William Burroughs, Allen Ginsberg, and Jack Kerouac all described \n
their experiences with ayahuasca, most famously in Burroughs's book The Yage Letters. """

In [7]:
model.predict(tu.keras_text_vectorizer(s, tokenizer))

array([[0.3476559 , 0.28875634, 0.36358783]], dtype=float32)